In [ ]:
import cv2
import json
import numpy as np
import skimage.draw
import matplotlib.pyplot as plt
from imgaug import augmenters as iaa

In [ ]:
# Set output area size for image
plt.rcParams['figure.figsize'] = (15.0, 15.0)

## Load Data

In [ ]:
img = cv2.imread("test.png")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
height, width = img.shape[:2]
print(width, height)
plt.imshow(img)
plt.show()

## Check Various Augument Image

In [ ]:
# aug = iaa.Affine(rotate=(-180, 180))
# aug = iaa.LogContrast(gain=(0.6,1.1))
# aug = iaa.Resize((0.5, 1.1))
aug = iaa.GaussianBlur(sigma=(0.0, 3.0))

In [ ]:
img_aug = aug.augment_image(img)
h, w = img_aug.shape[:2]
print(w, h)
plt.imshow(img_aug)
plt.show()

In [ ]:
seq = iaa.Sequential([
    iaa.Sometimes(0.8, iaa.Affine(rotate=(-180, 180))),
    iaa.Sometimes(0.8, iaa.LogContrast(gain=(0.6, 1.1))),
    iaa.Sometimes(0.8, iaa.Resize((0.5, 1.1))),
    iaa.Sometimes(0.8, iaa.GaussianBlur(sigma=(0.0, 3.0)))
])

img_aug = seq.augment_image(img)
plt.imshow(img_aug)
plt.show()

## Load annotation file

In [ ]:
annotations = json.load(open("test_label.json"))
annotations = list(annotations.values())
annotations = [a for a in annotations if a['regions']]

polygons = []
names = []
for a in annotations:
    polygons = [r['shape_attributes'] for r in a['regions']] 
    names = [r['region_attributes'] for r in a['regions']]

## Check masked data

In [ ]:
mask = np.zeros([height, width, len(polygons)], dtype=np.uint8)
mask_attr = []
plt.imshow(img)
plt.show()
for i, (p, n) in enumerate(zip(polygons, names)):
    rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
    mask[rr, cc, i] = 1
    
    print(n["name"])
    mask_attr.append(n["name"])
    plt.imshow(mask[:, :, i], cmap="gray")
    plt.show()

## PCA

In [ ]:
from math import atan2, cos, sin, sqrt, pi

In [ ]:
def drawAxis(img, p_, q_, colour, scale):
    p = list(p_)
    q = list(q_)
    
    angle = atan2(p[1] - q[1], p[0] - q[0]) # angle in radians
    hypotenuse = sqrt((p[1] - q[1]) * (p[1] - q[1]) + (p[0] - q[0]) * (p[0] - q[0]))
    # Here we lengthen the arrow by a factor of scale
    q[0] = p[0] - scale * hypotenuse * cos(angle)
    q[1] = p[1] - scale * hypotenuse * sin(angle)
    cv2.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), colour, 1, cv2.LINE_AA)
    # create the arrow hooks
    p[0] = q[0] + 9 * cos(angle + pi / 4)
    p[1] = q[1] + 9 * sin(angle + pi / 4)
    cv2.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), colour, 1, cv2.LINE_AA)
    p[0] = q[0] + 9 * cos(angle - pi / 4)
    p[1] = q[1] + 9 * sin(angle - pi / 4)
    cv2.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), colour, 1, cv2.LINE_AA)

In [ ]:
def getOrientation(pts, img):
    
    sz = len(pts)
    data_pts = np.empty((sz, 2), dtype=np.float64)
    for i in range(data_pts.shape[0]):
        data_pts[i,0] = pts[i,0,0]
        data_pts[i,1] = pts[i,0,1]
    # Perform PCA analysis
    mean = np.empty((0))
    mean, eigenvectors, eigenvalues = cv2.PCACompute2(data_pts, mean)
    # Store the center of the object
    cntr = (int(mean[0,0]), int(mean[0,1]))
    
    
    cv2.circle(img, cntr, 3, (255, 0, 255), 2)
    p1 = (cntr[0] + 0.02 * eigenvectors[0,0] * eigenvalues[0,0], cntr[1] + 0.02 * eigenvectors[0,1] * eigenvalues[0,0])
    p2 = (cntr[0] - 0.02 * eigenvectors[1,0] * eigenvalues[1,0], cntr[1] - 0.02 * eigenvectors[1,1] * eigenvalues[1,0])
    drawAxis(img, cntr, p1, (0, 255, 0), 1)
    drawAxis(img, cntr, p2, (255, 255, 0), 5)
    angle = atan2(eigenvectors[0,1], eigenvectors[0,0]) # orientation in radians
    
    return angle

In [ ]:
for i in range(mask.shape[-1]):
    ret, thresh = cv2.threshold(mask[:,:,i], 0.5, 1.0, cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for i, c in enumerate(contours):
        area = cv2.contourArea(c)
        if area < 1e2 or 1e5 < area:
            continue
            
        cv2.drawContours(img, contours, i, (0, 0, 255), 2)
        getOrientation(c, img)
        
plt.imshow(img)
plt.show()

## Augument image and mask

In [ ]:
# Integrate image numpy and mask numpy
img_and_mask = np.append(img, mask, axis=2)

# Check source image
plt.imshow(img_and_mask[:,:,:3])
plt.show()

# Augment
aug = iaa.Affine(rotate=(-180, 180))
test = aug.augment_image(img_and_mask)
test_img = test[:,:,:3]
test_mask = test[:,:,3:]

# Check augmented image
plt.imshow(test_img)
plt.show()

# Save augmented image
# cv2.imwrite("test_augment.png", cv2.cvtColor(test_img, cv2.COLOR_RGB2BGR))

# Make augmented masked label data
result = {}
result["test"] = {"filename":"test_augment.png", "file_attributes":{}, "size": 0}
regions = []
for i in range(test_mask.shape[-1]):
    attr = {}
    attr["region_attributes"] = {"name": mask_attr[i]}
    
    tmp = test_mask[:,:,i]
    ret, thresh = cv2.threshold(tmp, 0.5, 1.0, cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    contours_index = 0
    for k in range(len(contours)):
        if k != len(contours) - 1:
            if len(contours[k]) < len(contours[k + 1]):
                contours_index = k + 1
    
    contours = contours[contours_index]
    all_points_x = [int(contours[k][0][0]) for k in range(len(contours))]
    all_points_y = [int(contours[k][0][1]) for k in range(len(contours))]
    attr["shape_attributes"] = {"name": "polyline", "all_points_x": all_points_x, "all_points_y": all_points_y}
    regions.append(attr)
    plt.imshow(tmp, cmap="gray")
    plt.show()
    
result["test"]["regions"] = regions

In [ ]:
# Save json
with open("label.json", 'w') as f:
    json.dump(result, f)